In [1]:
import pandas as pd

/usr/local/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/local/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [13]:
NMF_results = pd.read_csv('NMF_1M.csv')
NMF_results.rename(columns = {'predicted_rating':'NMF_predicted_rating'}, inplace = True)
CoClustering_results = pd.read_csv('CoClustering_1M.csv')
CoClustering_results.rename(columns = {'predicted_rating':'CoClustering_predicted_rating'}, inplace = True)
SVD_results = pd.read_csv('SVD_1M.csv')
SVD_results.rename(columns = {'predicted_rating':'SVD_predicted_rating'}, inplace = True)
NN_results = pd.read_csv('NeuralNet_Prediction_1M.csv')
NN_results.rename(columns = {'predicted_rating':'NN_predicted_rating'}, inplace = True)
KNN_results = pd.read_csv('KNNBaseline_1M.csv')
KNN_results.rename(columns = {'predicted_rating':'KNN_predicted_rating'}, inplace = True)
Slopone_results = pd.read_csv('SlopeOne_1M.csv')
Slopone_results.rename(columns = {'predicted_rating':'SlopeOne_predicted_rating'}, inplace = True)
Ensemble_DF = NMF_results
Ensemble_DF['CoClustering_predicted_rating'] = CoClustering_results['CoClustering_predicted_rating']
Ensemble_DF['SVD_predicted_rating'] = SVD_results['SVD_predicted_rating']
Ensemble_DF['NN_predicted_rating'] = NN_results['NN_predicted_rating']
Ensemble_DF['SlopeOne_predicted_rating'] = Slopone_results['SlopeOne_predicted_rating']
Ensemble_DF['KNN_predicted_rating'] = KNN_results['KNN_predicted_rating']
Ensemble_DF['Ensemble_Rating'] = (Ensemble_DF['CoClustering_predicted_rating']+Ensemble_DF['NMF_predicted_rating']+Ensemble_DF['NN_predicted_rating']+2*Ensemble_DF['SVD_predicted_rating']+Ensemble_DF['SlopeOne_predicted_rating']+Ensemble_DF['KNN_predicted_rating'])/7
Ensemble_DF.head()

,userId,movieId,NMF_predicted_rating,true_rating,CoClustering_predicted_rating,SVD_predicted_rating,NN_predicted_rating,SlopeOne_predicted_rating,KNN_predicted_rating,Ensemble_Rating
0,1,1193,4.690972,5,4.888144,4.520033,5.180770,4.726890,4.600525,4.732481
1,1,3408,4.043078,4,4.439016,4.334860,4.093561,4.189577,4.184837,4.231398
2,1,919,4.557683,4,4.752106,4.775469,4.723738,4.568571,4.667060,4.688585
3,1,2797,4.151704,4,4.404727,4.091292,4.053844,4.149028,4.214397,4.165183
4,1,720,5.000000,3,4.946462,4.366757,5.180150,4.775051,4.659941,4.756446


In [14]:
from sklearn.metrics import mean_absolute_error
print(mean_absolute_error(Ensemble_DF['true_rating'], Ensemble_DF['Ensemble_Rating']))

0.6897525623082031


In [15]:
from sklearn.metrics import mean_squared_error
from math import sqrt

rmse = sqrt(mean_squared_error(Ensemble_DF['true_rating'], Ensemble_DF['Ensemble_Rating']))
rmse

0.8800719024872052

In [16]:
# all movie ids vs titles
df_movies = pd.read_csv('movies.csv')[['movieId', 'title']]
all_movies_dict = pd.Series(df_movies['title'].values,index=df_movies['movieId']).to_dict()

FileNotFoundError: File b'movies.csv' does not exist

In [75]:
# all users in test dataset 
all_users = Ensemble_DF['userId'].unique().tolist()

In [76]:
# results as per predicted ratings in test dataset 

df_prediction1 = Ensemble_DF[['userId', 'movieId', 'Ensemble_Rating']]
df_prediction1['predicted_movies'] = df_prediction1 \
  .apply(lambda x: (x['movieId'], x['Ensemble_Rating']), axis=1)

df_prediction2 = df_prediction1[['userId', 'predicted_movies']]

df_prediction_formatted = df_prediction2 \
  .groupby('userId')['predicted_movies'].apply(list).reset_index(name='recommendation')

df_prediction_sorted = df_prediction_formatted

df_prediction_sorted['recommendation'] = \
  df_prediction_sorted['recommendation'].apply( \
    lambda x: sorted(x, key=lambda tup: tup[1], reverse=True))
  
# df_prediction_sorted

sorted_reco_by_userid = pd.Series( \
  df_prediction_sorted['recommendation'].values,index=df_prediction_sorted['userId']).to_dict()

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [77]:
def get_top_n_recommendations(user, n=10):
  top_n_reco = sorted_reco_by_userid[user][:n]
  
  return [all_movies_dict[int(x[0])] for x in top_n_reco]

In [82]:
# results as per true ratings in test dataset

df_true1 = Ensemble_DF[['userId', 'movieId', 'true_rating']]
df_true1['movies'] = df_true1 \
  .apply(lambda x: (x['movieId'], x['true_rating']), axis=1)

df_true2 = df_true1[['userId', 'movies']]

df_true_formatted = df_true2 \
  .groupby('userId')['movies'].apply(list).reset_index(name='recommendation')

df_true_sorted = df_true_formatted

df_true_sorted['recommendation'] = \
  df_true_sorted['recommendation'].apply( \
    lambda x: sorted(x, key=lambda tup: tup[1], reverse=True))

# df_true_sorted

sorted_true_ratings_by_userid = pd.Series( \
  df_true_sorted['recommendation'].values,index=df_true_sorted['userId']).to_dict()

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [78]:
def get_top_n_true_ratings(user, n=10):
  top_n_true_ratings = sorted_true_ratings_by_userid[user][:n]
  
  return [all_movies_dict[int(x[0])] for x in top_n_true_ratings]

In [79]:
# output for each user in test dataset

user_out = []
i = 0
for user in all_users:
  out = []

  # user
  out.append(user)

  # top n recommendations (n = 10 by default)
  top_n_reco_for_user = get_top_n_recommendations(user)
  out.append(top_n_reco_for_user)

  top_n_true_ratings_for_user = get_top_n_true_ratings(user)
  true_positives = list(set(top_n_reco_for_user) & set(top_n_true_ratings_for_user))
  false_positives = list(set(top_n_reco_for_user) - set(top_n_true_ratings_for_user))
  false_negatives = list(set(top_n_true_ratings_for_user) - set(top_n_reco_for_user))

  # precision for user
  precision_for_user = len(true_positives) / float(len(true_positives) + len(false_positives))
  out.append(precision_for_user)

  # recall for user
  recall_for_user = len(true_positives) / float(len(true_positives) + len(false_negatives))
  out.append(recall_for_user)
  
  user_out.append(out)

In [80]:
# output for all users in test dataset

df_out = pd.DataFrame(user_out, columns=['userId', 'topNreco', 'precision_user', 'recall_user'])
df_out

,userId,topNreco,precision_user,recall_user
0,1,[Three Colors: Blue (Trois couleurs: Bleu) (19...,1.0,1.0
1,2,"[Room with a View, A (1986), Hackers (1995), L...",1.0,1.0
2,3,"[The Devil's Advocate (1997), Diabolique (1996)]",1.0,1.0
3,4,"[Fight Club (1999), Lord of the Rings: The Ret...",0.5,0.5
4,5,"[Fight Club (1999), Goodfellas (1990), When We...",0.7,0.7
5,6,"[Shawshank Redemption, The (1994), Pulp Fictio...",1.0,1.0
6,7,"[Rosemary's Baby (1968), About Schmidt (2002),...",1.0,1.0
7,8,"[One Fine Day (1996), My Fellow Americans (199...",1.0,1.0
8,10,"[Shawshank Redemption, The (1994), General, Th...",0.9,0.9
9,11,"[One Flew Over the Cuckoo's Nest (1975), Airpl...",1.0,1.0


## Final Results for ensemble

In [83]:
# EVALUATION

# mae
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(Ensemble_DF['true_rating'], Ensemble_DF['Ensemble_Rating'])
print('mae', mae)

# rmse
from sklearn.metrics import mean_squared_error
from math import sqrt
rmse = sqrt(mean_squared_error(Ensemble_DF['true_rating'], Ensemble_DF['Ensemble_Rating']))
print('rmse', rmse)

# precision
precision = df_out['precision_user'].mean()
print('avg precision', precision)

# recall
recall = df_out['recall_user'].mean()
print('avg recall', recall)

# f measure
f_measure = (2.0 * precision * recall) / (precision + recall)
print('f measure', f_measure)

mae 0.6145030511458718
rmse 0.8092740009973045
avg precision 0.845113408390796
avg recall 0.8451134895954507
f measure 0.8451134489931214
